In [11]:
from statsbombpy import sb
import pandas as pd

In [12]:
flat_events = sb.events(match_id=3795107, split=True, flatten_attrs=False)
passes = pd.DataFrame(flat_events['passes'])

In [13]:
passes['outcome'] = [p['outcome']['name'] if 'outcome' in p else 'Complete' for p in passes['pass']]
passes['recipient_id'] = [p['recipient']['id'] if 'recipient' in p else None for p in passes['pass']]
passes['recipient_name'] = [p['recipient']['name'] if 'recipient' in p else None for p in passes['pass']]
passes['location_end'] = [p['end_location'] if 'end_location' in p else None for p in passes['pass']]
passes['height'] = [p['height']['name'] if 'height' in p else None for p in passes['pass']]
passes['type'] = [p['type']['name'] if 'type' in p else None for p in passes['pass']]
passes['body_part'] = [p['body_part']['name'] if 'body_part' in p else None for p in passes['pass']]
passes['location_x'] = [p[0] for p in passes['location']]
passes['location_y'] = [p[1] for p in passes['location']]
passes['location_end_x'] = [p[0] for p in passes['location_end']]
passes['location_end_y'] = [p[1] for p in passes['location_end']]

In [14]:
[p for p in passes['pass']][0:5]

[{'recipient': {'id': 5642, 'name': 'Axel Witsel'},
  'length': 6.87968,
  'angle': -2.5737486,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [54.2, 36.3],
  'type': {'id': 65, 'name': 'Kick Off'},
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 3077, 'name': 'Jan Vertonghen'},
  'length': 23.700844,
  'angle': -2.3890185,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [34.1, 17.5],
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 5632, 'name': 'Thorgan Hazard'},
  'length': 15.401948,
  'angle': -1.0170146,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [41.1, 1.6],
  'body_part': {'id': 38, 'name': 'Left Foot'}},
 {'recipient': {'id': 3509, 'name': 'Thibaut Courtois'},
  'length': 42.694847,
  'angle': 2.407559,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [5.4, 30.9],
  'body_part': {'id': 40, 'name': 'Right Foot'}},
 {'recipient': {'id': 20005, 'name': 'Toby Alderwe

In [15]:
passes.iloc[0]

id                                 474c5607-72d6-4d59-889f-5724d4698f48
index                                                                 5
period                                                                1
timestamp                                                  00:00:01.215
minute                                                                0
second                                                                1
type                                                           Kick Off
possession                                                            2
possession_team                                                 Belgium
play_pattern                                              From Kick Off
team                                                            Belgium
player                                             Romelu Lukaku Menama
position                                                 Center Forward
location                                                   [60.0

In [16]:
passes.iloc[0]['pass']

{'recipient': {'id': 5642, 'name': 'Axel Witsel'},
 'length': 6.87968,
 'angle': -2.5737486,
 'height': {'id': 1, 'name': 'Ground Pass'},
 'end_location': [54.2, 36.3],
 'type': {'id': 65, 'name': 'Kick Off'},
 'body_part': {'id': 40, 'name': 'Right Foot'}}

In [17]:
# Omit columns we don't care about
passes = passes.drop(columns=[
    'related_events',
    'pass',
    'location',
    'location_end',
    'timestamp',
    'second',
    'possession_team_id',
    'possession_team'
])

In [18]:
# Convert recipient id to int so it gets written to csv correctly
passes = passes.astype({'recipient_id': 'Int64'})

In [19]:
# Get 360 frames
frames = sb.competition_frames(
    country="Europe",
    division= "UEFA Euro",
    season="2020"
)

In [20]:
frames = frames[frames["match_id"] == 3795107]
frames = frames.drop(columns=["visible_area"])
frames

,event_uuid,freeze_frame,match_id
6654,474c5607-72d6-4d59-889f-5724d4698f48,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
6655,8b86747e-8a9b-4fdc-853b-81c4226a61ad,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
6656,283cd6e0-bd36-44f8-b8bb-5f4cfeec9850,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
6657,d321c735-f50a-47ce-bcb5-3e28a48e6e09,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
6658,655fe363-2574-455e-8e8d-1bdd88964f28,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
...,...,...,...
9809,8b5d9f82-7c7e-42df-b0fa-959f3740dc5e,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107
9810,5d4ca016-8d47-47c1-9855-03a7c0d2f5cb,"[{'teammate': False, 'actor': False, 'keeper':...",3795107
9811,d018c006-2413-45dd-b453-a8d4e64b3b60,"[{'teammate': False, 'actor': False, 'keeper':...",3795107
9812,9257d9ca-ae9b-40cf-8d72-fd40c5d4e785,"[{'teammate': True, 'actor': False, 'keeper': ...",3795107


In [57]:
# Use frames data to update location_x and location_y in passes
frames.iloc[1]['event_uuid']
for (_, frame) in frames.iterrows():
    uuid = frame['event_uuid']
    freeze = frame['freeze_frame']
    actor = next((x for x in freeze if x['actor']), None)
    location = actor['location']
    row = passes.index[passes['id'] == uuid]
    if len(row) > 0:
        passes.iat[row[0], passes.columns.get_loc('location_x')] = location[0]
        passes.iat[row[0], passes.columns.get_loc('location_y')] = location[1]

In [58]:
# Write to csv
frames.to_csv('data/frames.csv', index=False)
passes.to_csv('data/passes.csv', index=False)